In [6]:
%cd ..
import os
os.getcwd()

/Users/macbook/Documents/Code/LLMdesignedEA-comp


'/Users/macbook/Documents/Code/LLMdesignedEA-comp'

In [1]:
import random
import numpy as np

np.random.uniform(5, 10, 5)

array([5.55958355, 8.69108802, 5.10764078, 7.0700668 , 7.24751518])

In [3]:
from ioh import get_problem, logger, problem

z = get_problem(1, 1, 5)
print(z)


<RealSingleObjectiveProblem 1. Sphere (iid=1 dim=5)>


In [1]:
import os
from scipy.io import loadmat
from codes.gnbg_python.GNBG_instances import GNBG
from LLaMEA.misc.utils import aoc_logger, correct_aoc, OverBudgetException

problem_idx_to_load = 5
filename = f'f{problem_idx_to_load}.mat'

GNBG_tmp = loadmat(os.path.join("codes/gnbg_python", filename))['GNBG']
MaxEvals = np.array([item[0] for item in GNBG_tmp['MaxEvals'].flatten()])[0, 0]
AcceptanceThreshold = np.array([item[0] for item in GNBG_tmp['AcceptanceThreshold'].flatten()])[0, 0]
Dimension = np.array([item[0] for item in GNBG_tmp['Dimension'].flatten()])[0, 0]
CompNum = np.array([item[0] for item in GNBG_tmp['o'].flatten()])[0, 0]  # Number of components
MinCoordinate = np.array([item[0] for item in GNBG_tmp['MinCoordinate'].flatten()])[0, 0]
MaxCoordinate = np.array([item[0] for item in GNBG_tmp['MaxCoordinate'].flatten()])[0, 0]
CompMinPos = np.array(GNBG_tmp['Component_MinimumPosition'][0, 0])
CompSigma = np.array(GNBG_tmp['ComponentSigma'][0, 0], dtype=np.float64)
CompH = np.array(GNBG_tmp['Component_H'][0, 0])
Mu = np.array(GNBG_tmp['Mu'][0, 0])
Omega = np.array(GNBG_tmp['Omega'][0, 0])
Lambda = np.array(GNBG_tmp['lambda'][0, 0])
RotationMatrix = np.array(GNBG_tmp['RotationMatrix'][0, 0])
OptimumValue = np.array([item[0] for item in GNBG_tmp['OptimumValue'].flatten()])[0, 0]
OptimumPosition = np.array(GNBG_tmp['OptimumPosition'][0, 0])
gnbg = GNBG(MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate, MaxCoordinate, CompMinPos, CompSigma, CompH, Mu, Omega, Lambda, RotationMatrix, OptimumValue, OptimumPosition)


ModuleNotFoundError: No module named 'codes'

In [10]:
gnbg.MinCoordinate

-100

In [22]:
random_solution_vector = np.random.uniform(
    gnbg.MinCoordinate,
    gnbg.MaxCoordinate,
    gnbg.Dimension
)

# The GNBG.fitness method expects a 2D array (batch of solutions)
# and returns a 1D array of fitnesses.
fitness = gnbg.fitness(random_solution_vector.reshape(1, -1))
fitness

GNBG.fitness called with X shape: (1, 30), self.FE: 5, self.MaxEvals: 500000
AOCC: 0.49730865970463295


array([-334.13145235])

In [23]:
gnbg.FEhistory

array([-334.03726578, -333.94262777, -333.97042657, -334.01825872,
       -334.05542986, -334.13145235])

In [ ]:
import test.calc_aocc_from_gnbg as calc_aocc_from_gnbg
import importlib
importlib.reload(calc_aocc_from_gnbg)

from test.calc_aocc_from_gnbg import calculate_aocc_from_gnbg_history
calculate_aocc_from_gnbg_history(fe_history=gnbg.FEhistory, optimum_value=gnbg.OptimumValue, budget_B=gnbg.MaxEvals)

0.1471398644512762

In [13]:
nums = [10, 20, 30, 40, 50]
sample = random.sample(nums, 3)
print(sample)

[20, 30, 40]


In [14]:
arr = np.array([-2, -0.5, 0.3, 0.9, 1.5, 3])
clipped = np.clip(arr, 0, 1)
print(clipped)

[0.  0.  0.3 0.9 1.  1. ]


In [ ]:
f random.random() < self.mut_prob:
            mutation_values = np.random.normal(0, self.mut_strength, self.dimension)
            mutated_individual = individual + mutation_values
            mutated_individual = np.clip(mutated_individual, self.lower_bounds, self.upper_bounds)
            return mutated_individual
        return individual.copy()

In [11]:
import random

dims = 30
def create_random_bounds_list(num_components=dims, min_val=-10, max_val=10):
    """
    Creates a list of random [a, b] bounds.

    Args:
        num_components (int): The number of [a, b] pairs to generate.
        min_val (int): The minimum possible value for 'a' and 'b'.
        max_val (int): The maximum possible value for 'a' and 'b'.

    Returns:
        list: A list of lists, e.g., [[-5, 2], [0, 8], ...]
    """
    bounds_list = []
    for _ in range(num_components):
        a = random.randint(min_val, max_val -1) # Ensure 'a' can be less than 'max_val' to allow 'b' to be 'max_val'
        b = random.randint(a, max_val)         # Ensure b >= a
        bounds_list.append([a, b])
    return bounds_list

# Generate the list of 30 random bounds
random_bounds = create_random_bounds_list(30)

# Print the generated list (optional)
# for i, bound_pair in enumerate(random_bounds):
#     print(f"Component {i+1}: {bound_pair}")

print(random_bounds)

[[-7, 4], [-10, 1], [2, 9], [-9, 7], [5, 6], [3, 4], [1, 5], [-3, -3], [8, 10], [5, 5], [8, 10], [7, 9], [6, 10], [-10, -8], [3, 4], [-10, 1], [-7, -1], [5, 7], [-8, -3], [-10, -10], [-5, 10], [5, 5], [-3, 2], [3, 5], [5, 6], [9, 9], [5, 7], [2, 4], [-1, 6], [4, 4]]


In [12]:
from independent_obj_func import sphere_objective_for_rcga
from simpleGA import SimpleRCGA

rcga = SimpleRCGA(objective_function=sphere_objective_for_rcga,
                  dimension=dims, bounds=random_bounds)

In [13]:
individuals = rcga._create_individual()

rcga.optimize(50)

(array([ -3.32517811,  -1.38631044,   7.86179129,  -1.45014865,
          5.50101485,   3.60650417,   3.01768394,  -3.        ,
          8.69708867,   5.        ,   8.69337661,   7.42466195,
          9.0060345 ,  -9.19859381,   3.24411347,  -1.08430913,
         -4.43984574,   6.81120034,  -3.7519164 , -10.        ,
         -0.68945411,   5.        ,  -0.34338907,   3.00795517,
          5.45142659,   9.        ,   6.85004364,   2.1881757 ,
          4.95534246,   4.        ]),
 np.float64(964.8336457164237))

In [ ]:
import numpy as np
from scipy.io import loadmat
import os
import logging
from typing import Optional, Tuple, List

class HybridPSO_NelderMead:
    def __init__(self, budget: int, dim: int, lower_bounds: list[float], upper_bounds: list[float]):
        self.budget = int(budget)
        self.dim = int(dim)
        self.lower_bounds = np.array(lower_bounds, dtype=float)
        self.upper_bounds = np.array(upper_bounds, dtype=float)

        self.eval_count = 0
        self.best_solution_overall = np.random.uniform(self.lower_bounds, self.upper_bounds, self.dim)
        self.best_fitness_overall = float('inf')

        self.population_size = 50
        self.population = None
        self.velocities = None
        self.personal_bests = None
        self.personal_best_fitness = None
        self.global_best = None
        self.inertia_weight = 0.7
        self.cognitive_coefficient = 1.4
        self.social_coefficient = 1.4


    def optimize(self, objective_function: callable) -> tuple:
        self.eval_count = 0
        self.population = np.random.uniform(self.lower_bounds, self.upper_bounds, (self.population_size, self.dim))
        self.velocities = np.zeros_like(self.population)
        self.personal_bests = self.population.copy()
        self.personal_best_fitness = np.full(self.population_size, np.inf)
        self.global_best = None

        fitness_values = objective_function(self.population)
        self.eval_count += self.population_size
        
        for i, fitness in enumerate(fitness_values):
            if fitness < self.personal_best_fitness[i]:
                self.personal_best_fitness[i] = fitness
                self.personal_bests[i] = self.population[i].copy()
            if fitness < self.best_fitness_overall:
                self.best_fitness_overall = fitness
                self.best_solution_overall = self.population[i].copy()
                self.global_best = self.population[i].copy()

        while self.eval_count < self.budget:
            self.update_velocities()
            self.update_positions()
            fitness_values = objective_function(self.population)
            self.eval_count += self.population_size
            for i, fitness in enumerate(fitness_values):
                if fitness < self.personal_best_fitness[i]:
                    self.personal_best_fitness[i] = fitness
                    self.personal_bests[i] = self.population[i].copy()
                if fitness < self.best_fitness_overall:
                    self.best_fitness_overall = fitness
                    self.best_solution_overall = self.population[i].copy()
                    self.global_best = self.population[i].copy()

            #Nelder-Mead local search
            self.nelder_mead_local_search(objective_function)


        optimization_info = {
            'function_evaluations_used': self.eval_count,
            'final_best_fitness': self.best_fitness_overall
        }

        return self.best_solution_overall, self.best_fitness_overall, optimization_info


    def update_velocities(self):
        r1 = np.random.rand(self.population_size, self.dim)
        r2 = np.random.rand(self.population_size, self.dim)
        self.velocities = self.inertia_weight * self.velocities + \
                          self.cognitive_coefficient * r1 * (self.personal_bests - self.population) + \
                          self.social_coefficient * r2 * (self.global_best - self.population)

    def update_positions(self):
        self.population = self.population + self.velocities
        self.population = np.clip(self.population, self.lower_bounds, self.upper_bounds)


    def nelder_mead_local_search(self, objective_function):
        if self.eval_count < self.budget:
            simplex = np.stack([self.best_solution_overall] + [np.random.uniform(self.lower_bounds, self.upper_bounds, self.dim) for _ in range(self.dim)])
            
            for _ in range(100): # Limit iterations to avoid excessive local search time
                if self.eval_count >= self.budget:
                    break
                fitness_values = objective_function(simplex)
                self.eval_count += simplex.shape[0]
                #Simple Nelder Mead implementation, can be improved
                sorted_indices = np.argsort(fitness_values)
                best_point = simplex[sorted_indices[0]]
                worst_point = simplex[sorted_indices[-1]]
                centroid = np.mean(simplex[sorted_indices[:-1]], axis=0)
                reflection_point = 2 * centroid - worst_point
                reflection_fitness = objective_function(reflection_point.reshape(1, -1))
                self.eval_count += 1

                if reflection_fitness < self.best_fitness_overall:
                    self.best_fitness_overall = reflection_fitness
                    self.best_solution_overall = reflection_point.copy()
                    simplex[sorted_indices[-1]] = reflection_point
                else:
                    break # Simplified termination condition
def run_optimization(MaxEvals, AcceptanceThreshold, 
                     Dimension, CompNum, MinCoordinate, MaxCoordinate,
                     CompMinPos, CompSigma, CompH, Mu, Omega,
                     Lambda, RotationMatrix, OptimumValue, OptimumPosition,
                    num_runs: int = 31,
                    seed: Optional[int] = None) -> Tuple[List[float], List[np.ndarray]]:
    """
    Run multiple optimization runs for a given problem
    
    Args:
        problem_index: GNBG problem index (1-24)
        num_runs: Number of independent runs
        seed: Random seed for reproducibility
        
    Returns:
        Tuple of (best_fitness_values, best_solutions)
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Load problem instance
    from codes.gnbg_python.GNBG_instances import GNBG
    gnbg = GNBG(MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate, MaxCoordinate, CompMinPos, CompSigma, CompH, Mu, Omega, Lambda, RotationMatrix, OptimumValue, OptimumPosition)
    # Set up bounds
    bounds = (gnbg.MinCoordinate, gnbg.MaxCoordinate)
    
    # Initialize results storage
    best_values = []
    best_solutions = []
    
    for run in range(num_runs):
        logging.info(f"Starting run {run + 1}/{num_runs}")
        
        # Initialize algorithm
       
        
        optimizer = HybridPSO_NelderMead(
            budget=gnbg.MaxEvals,
            dim=gnbg.Dimension,
            lower_bounds=[gnbg.MinCoordinate for _ in range(gnbg.Dimension)],
            upper_bounds=[gnbg.MaxCoordinate for _ in range(gnbg.Dimension)]
        )
        
        # Run optimization
        best_solution, best_fitness, _ = optimizer.optimize(
            objective_function=gnbg.fitness
        )
        
        best_values.append(best_fitness)
        best_solutions.append(best_solution)
        
        logging.info(f"Run {run + 1} completed. Best fitness: {best_fitness}")
    
    return best_values, best_solutions
if __name__ == "__main__":
    folder_path = "codes/gnbg_python"
    # Example usage
    ProblemIndex = 2
    if 1 <= ProblemIndex <= 24:
        
        filename = f'f{ProblemIndex}.mat'
        GNBG_tmp = loadmat(os.path.join(folder_path, filename))['GNBG']
        MaxEvals = np.array([item[0] for item in GNBG_tmp['MaxEvals'].flatten()])[0, 0]
        AcceptanceThreshold = np.array([item[0] for item in GNBG_tmp['AcceptanceThreshold'].flatten()])[0, 0]
        Dimension = np.array([item[0] for item in GNBG_tmp['Dimension'].flatten()])[0, 0]
        CompNum = np.array([item[0] for item in GNBG_tmp['o'].flatten()])[0, 0]  # Number of components
        MinCoordinate = np.array([item[0] for item in GNBG_tmp['MinCoordinate'].flatten()])[0, 0]
        MaxCoordinate = np.array([item[0] for item in GNBG_tmp['MaxCoordinate'].flatten()])[0, 0]
        CompMinPos = np.array(GNBG_tmp['Component_MinimumPosition'][0, 0])
        CompSigma = np.array(GNBG_tmp['ComponentSigma'][0, 0], dtype=np.float64)
        CompH = np.array(GNBG_tmp['Component_H'][0, 0])
        Mu = np.array(GNBG_tmp['Mu'][0, 0])
        Omega = np.array(GNBG_tmp['Omega'][0, 0])
        Lambda = np.array(GNBG_tmp['lambda'][0, 0])
        RotationMatrix = np.array(GNBG_tmp['RotationMatrix'][0, 0])
        OptimumValue = np.array([item[0] for item in GNBG_tmp['OptimumValue'].flatten()])[0, 0]
        OptimumPosition = np.array(GNBG_tmp['OptimumPosition'][0, 0])
        
        best_values, best_solutions = run_optimization(MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate, MaxCoordinate,
                                                       CompMinPos, CompSigma, CompH, Mu, Omega, Lambda, RotationMatrix, OptimumValue, OptimumPosition)
    else:
        raise ValueError('ProblemIndex must be between 1 and 24.')
    
    print(f"\nResults for Problem {ProblemIndex}:")
    print(f"Best solution: {best_solutions}")

    print(f"Best fitness values: {best_values}")
    print(f"Mean fitness: {np.mean(best_values)}")
    print(f"Std fitness: {np.std(best_values)}") 

In [ ]:
auc_mean = 0
auc_std = 0
detailed_aucs = [0, 0, 0] # 3 function groups of GNBG benchmark, 3 detaiteed fedback       code = solution.code
algorithm_name = solution.name
code = solution.code
print("Gonna execute")
exec(code, globals()) # extract the code part inside the string, ex exec("a = 3 + 4") -> print(a) -> 7
print("Finish execute")
error = ""

aucs = []
detail_aucs = []
algorithm = None
for dim in [5]:
    for fid in np.arange(1, 25): # cal 24 functions from GNBG
        filename = f'f{fid}.mat'
        GNBG_tmp = loadmat(os.path.join("codes/gnbg_python", filename))['GNBG']
        MaxEvals = np.array([item[0] for item in GNBG_tmp['MaxEvals'].flatten()])[0, 0]
        AcceptanceThreshold = np.array([item[0] for item in GNBG_tmp['AcceptanceThreshold'].flatten()])[0, 0]
        Dimension = np.array([item[0] for item in GNBG_tmp['Dimension'].flatten()])[0, 0]
        CompNum = np.array([item[0] for item in GNBG_tmp['o'].flatten()])[0, 0]  # Number of components
        MinCoordinate = np.array([item[0] for item in GNBG_tmp['MinCoordinate'].flatten()])[0, 0]
        MaxCoordinate = np.array([item[0] for item in GNBG_tmp['MaxCoordinate'].flatten()])[0, 0]
        CompMinPos = np.array(GNBG_tmp['Component_MinimumPosition'][0, 0])
        CompSigma = np.array(GNBG_tmp['ComponentSigma'][0, 0], dtype=np.float64)
        CompH = np.array(GNBG_tmp['Component_H'][0, 0])
        Mu = np.array(GNBG_tmp['Mu'][0, 0])
        Omega = np.array(GNBG_tmp['Omega'][0, 0])
        Lambda = np.array(GNBG_tmp['lambda'][0, 0])
        RotationMatrix = np.array(GNBG_tmp['RotationMatrix'][0, 0])
        OptimumValue = np.array([item[0] for item in GNBG_tmp['OptimumValue'].flatten()])[0, 0]
        OptimumPosition = np.array(GNBG_tmp['OptimumPosition'][0, 0])
        problem = GNBG(MaxEvals, AcceptanceThreshold, Dimension, CompNum, MinCoordinate, MaxCoordinate, CompMinPos, CompSigma, CompH, Mu, Omega, Lambda, RotationMatrix, OptimumValue, OptimumPosition)

        print("Run or not!")
        for rep in range(5):
            np.random.seed(rep)
            try:
                print("Create the algorithm")
                algorithm = globals()[algorithm_name](budget=problem.MaxEvals, dim=problem.Dimension,
                    lower_bounds = [problem.MinCoordinate] * problem.Dimension,
                    upper_bounds = [problem.MaxCoordinate] * problem.Dimension
                    )
                print("Optimize the algorithm")
                algorithm.optimize(fitness_func=problem.fitness, 
        acceptance_threshold=problem.AcceptanceThreshold)
                print("Successful run the algo")
            except Exception:
                print("Can not run the algorithm")
            auc = calculate_aocc_from_gnbg_history(fe_history=problem.FEhistory,
                                                    optimum_value=problem.OptimumValue, 
                                                budget_B=problem.MaxEvals
                                            )

[array([-701.38228397]),
 array([-701.40752683]),
 array([-701.39165171]),
 array([-701.39545006]),
 array([-701.39219278]),
 array([-701.40072997]),
 array([-701.39465231]),
 array([-701.41351582]),
 array([-701.39316644]),
 array([-701.39600691]),
 array([-701.40612066]),
 array([-701.40616876]),
 array([-701.40055886]),
 array([-701.40576515]),
 array([-701.39422289]),
 array([-701.39859207]),
 array([-701.38671566]),
 array([-701.41207913]),
 array([-701.39848217]),
 array([-701.39516402]),
 array([-701.40336443]),
 array([-701.38904104]),
 array([-701.40554802]),
 array([-701.37800142]),
 array([-701.39489238]),
 array([-701.40109902]),
 array([-701.39335988]),
 array([-701.39453926]),
 array([-701.38891138]),
 array([-701.38744312])]

In [ ]:
# f1, f2, ...f24

# divided into 5 function groups;
# just ex: 
# seperate function: f1, f2, ..f5 
# functions with low or moderate conditioning: f6,..., f10;
# calculating AOCC

# ....

# each function will run through 3 instances, 3 independent run perfunction -> 9 run



In [5]:
from ioh import get_problem
get_problem(fid=1, instance=2, dimension=5)

<RealSingleObjectiveProblem 1. Sphere (iid=2 dim=5)>

In [10]:
from scipy.io import loadmat
data = loadmat("codes/gnbg_python/f1.mat")

In [ ]:
gnbg_tmp_data = data['GNBG']

dtype([('MaxEvals', 'O'), ('AcceptanceThreshold', 'O'), ('Dimension', 'O'), ('o', 'O'), ('MinCoordinate', 'O'), ('MaxCoordinate', 'O'), ('MinRandOptimaPos', 'O'), ('MaxRandOptimaPos', 'O'), ('MinExclusiveRange', 'O'), ('MaxExclusiveRange', 'O'), ('ComponentPositioningMethod', 'O'), ('Component_MinimumPosition', 'O'), ('MinSigma', 'O'), ('MaxSigma', 'O'), ('ComponentSigma', 'O'), ('H_pattern', 'O'), ('H_value', 'O'), ('Component_H', 'O'), ('MaxMu', 'O'), ('MinMu', 'O'), ('MaxOmega', 'O'), ('MinOmega', 'O'), ('localModalitySymmetry', 'O'), ('Mu', 'O'), ('Omega', 'O'), ('MaxLambda', 'O'), ('MinLambda', 'O'), ('LambdaConfigMethod', 'O'), ('LambdaValue4ALL', 'O'), ('lambda', 'O'), ('MinAngle', 'O'), ('MaxAngle', 'O'), ('Rotation', 'O'), ('RotationMatrix', 'O'), ('OptimumValue', 'O'), ('OptimumPosition', 'O'), ('FEhistory', 'O'), ('FE', 'O'), ('AcceptanceReachPoint', 'O'), ('BestFoundResult', 'O')])

In [14]:
def get_param(data_dict, key, is_scalar=True, dtype=None):
    val_array = data_dict[key].flatten()
    if not val_array.size: return None
    val = val_array[0]
    if is_scalar and isinstance(val, np.ndarray) and val.ndim >=0 : # Handle 0-dim arrays from MATLAB
        # Check if it's a scalar within a 1x1 array or just a scalar array
        if val.size == 1:
                val = val.item() # Get Python scalar
        # else it's an array meant to be returned as such, but flatten took [0]
        # This part needs to be careful based on .mat structure.
        # For GNBG, most scalars are wrapped like [[value]].
        elif val.ndim == 2 and val.shape[0] ==1 and val.shape[1] == 1:
                val = val[0,0]

    if isinstance(val, np.ndarray) and not is_scalar : # if it's meant to be an array
            pass # val is already an array
    elif is_scalar and not isinstance(val, np.ndarray): # it's already a python scalar
            pass

    if dtype: return np.array(val, dtype=dtype) # Ensure it's np.array if not scalar
    return np.array(val) if not isinstance(val, (int, float, np.number)) else val


In [18]:
get_param(gnbg_tmp_data, 'MaxEvals')
problem_accept_thresh = float(get_param(gnbg_tmp_data, 'AcceptanceThreshold'))
problem_dimension = int(get_param(gnbg_tmp_data, 'Dimension'))
problem_comp_num = int(get_param(gnbg_tmp_data, 'o'))
problem_min_coord = float(get_param(gnbg_tmp_data, 'MinCoordinate'))
problem_max_coord = float(get_param(gnbg_tmp_data, 'MaxCoordinate'))

problem_comp_min_pos = get_param(gnbg_tmp_data, 'Component_MinimumPosition', is_scalar=False)
problem_comp_sigma_flat = get_param(gnbg_tmp_data, 'ComponentSigma', is_scalar=False, dtype=np.float64)
problem_comp_sigma = problem_comp_sigma_flat.flatten() if problem_comp_sigma_flat.ndim > 0 else np.array([problem_comp_sigma_flat.item()])


problem_comp_h = get_param(gnbg_tmp_data, 'Component_H', is_scalar=False)
problem_mu = get_param(gnbg_tmp_data, 'Mu', is_scalar=False)
problem_omega = get_param(gnbg_tmp_data, 'Omega', is_scalar=False)

problem_lambda_flat = get_param(gnbg_tmp_data, 'lambda', is_scalar=False)
problem_lambda = problem_lambda_flat.flatten() if problem_lambda_flat.ndim > 0 else np.array([problem_lambda_flat.item()])


problem_rot_matrix = get_param(gnbg_tmp_data, 'RotationMatrix', is_scalar=False)
problem_opt_value = float(get_param(gnbg_tmp_data, 'OptimumValue'))
problem_opt_pos = get_param(gnbg_tmp_data, 'OptimumPosition', is_scalar=False)

In [21]:
problem_accept_thresh

1e-08

In [ ]:
import numpy as np
from scipy.optimize import differential_evolution

def sphere(x):
    return sum(xi**2 for xi in x)

bounds = [(-5, 5)] * 3
result = differential_evolution(sphere, bounds)

print("Best solution:", result.x)
print("Function value:", result.fun)

# result = differential_evolution(func, bounds, args=(), strategy='best1bin', 
#                                 maxiter=1000, popsize=15, tol=0.01, 
#                                 mutation=(0.5, 1), recombination=0.7, seed=None, 
#                                 callback=None, disp=False, polish=True, 
#                                 init='latinhypercube', atol=0)

# Key Parameters:
# func: The objective function to be minimized.

# bounds: A list of tuples specifying the lower and upper bounds for each parameter.

# strategy: The differential evolution strategy to use (e.g., 'best1bin').

# maxiter: Maximum number of generations.

# popsize: Population size (total population is popsize * len(bounds)).

# mutation: Mutation constant or range.

# recombination: Crossover probability.

# polish: If True, uses L-BFGS-B to refine the best solution.

# init: Method to initialize the population ('latinhypercube', 'random', or array).

# seed: Random seed for reproducibility.

Best solution: [0. 0. 0.]
Function value: 0.0


In [23]:
from LLaMEA.llamea.llm import Ollama_LLM

llm = Ollama_LLM(model="codellama:7b")

In [25]:
messages = [
        {"role": "user", "content": "Explain how a black hole forms in simple terms."}
    ]

reply = llm.query(messages)
print("LLM Response:\n", reply)

2025-05-23 19:12:13 INFO HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


LLM Response:
 A  black hole is formed when a massive star collapses into a singularity. The star's gravity eventually becomes so strong that it warps space and time around the star, creating a boundary called the event horizon. Once something crosses the event horizon, there is no escape from the black hole's gravitational pull. Any matter or energy that gets too close to the black hole will be pulled in and crushed out of existence.




In [ ]:
# LLAMEA
# initialize -> update_best -> 